In [1]:
! pip install py7zr pandas


   ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
   ---------------------------------------- 69.7/69.7 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/369.1 kB ? eta -:--:--
   ------------------------------------ --- 337.9/369.1 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 369.1/369.1 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.8 MB 5.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 6.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.8 MB 6.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/1.8 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.5/1.8 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.6 kB ? eta -:--:--
   -----------------------

In [2]:
import requests
import pandas as pd
import numpy as np
import gzip
import py7zr
import io
import seaborn as sns

In [3]:
#Fonction pour récupérer les données d'un fichier csv.gz ( pour les données météorologiques )
def recuperer_donnees_gz(dataset_id, fichier_csv, sep):
    """
    Cette fonction télécharge les données d'un fichier CSV.GZ à partir d'une URL et les charge dans un DataFrame pandas.
    
    Paramètres:
    dataset_id (str): identifiant du fichier CSV.GZ.
    fichier_csv (str): Le nom du fichier CSV.GZ à enregistrer localement.
    sep ( str ) : le séparateur à utiliser
    
    Retourne:
    DataFrame: Un DataFrame pandas contenant les données du fichier CSV.
    """

    # URL de base pour accéder à l'API
    base_api = "https://www.data.gouv.fr/api/1/"

    # Chemin pour accéder aux enregistrements du dataset
    key_api = "datasets/r/"

    # Construction de l'URL complète
    url = f"{base_api}{key_api}{dataset_id}"


    response = requests.get(url)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        df = pd.read_csv(gz, sep = sep)
    
    return df

## DAILY SWI

In [4]:
quot_25 = recuperer_donnees_gz('92065ec0-ea6f-4f5e-8827-4344179c0a7f', 'QUOT_2020-2025', ';')
quot_25.tail()

,LAMBX,LAMBY,DATE,PRENEI,PRELIQ,T,FF,Q,DLI,SSI,...,RESR_NEIGE6,HTEURNEIGE,HTEURNEIGE6,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H
21079847,11960,17450,20251027,0.0,0.0,15.5,2.4,6.076,2695.1,1376.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,12.1,20.3
21079848,11960,17450,20251028,0.0,0.1,15.4,2.1,7.500,2877.0,1195.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,11.2,22.5
21079849,11960,17450,20251029,0.0,1.0,17.0,2.9,9.242,3156.7,641.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,12.0,22.7
21079850,11960,17450,20251030,0.0,3.3,17.7,1.6,11.344,3426.6,178.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.205,0.0,16.5,18.9
21079851,11960,17450,20251031,0.0,0.4,17.9,1.6,11.022,3174.7,859.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.207,0.0,16.0,21.5


In [32]:
liste_id=['eb0d6e42-cee6-4d7c-bc5b-646be4ced72e','33417617-c0dd-4513-804e-c3f563cb81b4','08ad5936-cb9e-4284-a6fc-36b29aca9607',
          'ad584d65-7d2d-4ff1-bc63-4f93357ed196','10d2ce77-5c3b-44f8-bb46-4df27ed48595','da6cd598-498b-4e39-96ea-fae89a4a8a46',
          '92065ec0-ea6f-4f5e-8827-4344179c0a7f','adcca99a-6db0-495a-869f-40c888174a57']
# année=[1960-1969,1970-1979,1980-1989,'1990-1999',2000-2009,2010-2019,2020-202510,20251001-20251107]

In [ ]:
#Fusion des tables
liste_df=[]
for id in liste_id:
    data = recuperer_donnees_gz(id, '', ';')
    liste_df.append(data)
daily_swi_df=pd.concat(liste_df,ignore_index=True)
daily_swi_df.shape(

)
# Ce code sort une erreur de mémoire, les bases sont trop lourdes pour pouvoir toutes les fusionnées.

MemoryError: Unable to allocate 7.00 GiB for an array with shape (26, 36135476) and data type float64

In [ ]:
# Création d'une colonne YEAR
quot_25["YEAR"] = quot_25["DATE"].astype(str).str[:4].astype(int)

In [ ]:
quot_25[quot_25["YEAR"]==2020]["LAMBX"].unique()

134

In [25]:
quot_25[quot_25["YEAR"]==2021]["LAMBX"].unique()

array([  600,   760,   840,   920,  1000,  1080,  1160,  1240,  1320,
        1400,  1480,  1560,  1640,  1720,  1800,  1880,  1960,  2040,
        2120,  2200,  2280,  2360,  2440,  2520,  2600,  2680,  2760,
        2840,  2920,  3000,  3080,  3160,  3240,  3320,  3400,  3480,
        3560,  3640,  3720,  3800,  3880,  3960,  4040,  4120,  4200,
        4280,  4360,  4440,  4520,  4600,  4680,  4760,  4840,  4920,
        5000,  5080,  5160,  5240,  5320,  5400,  5480,  5560,  5640,
        5720,  5800,  5880,  5960,  6040,  6120,  6200,  6280,  6360,
        6440,  6520,  6600,  6680,  6760,  6840,  6920,  7000,  7080,
        7160,  7240,  7320,  7400,  7480,  7560,  7640,  7720,  7800,
        7880,  7960,  8040,  8120,  8200,  8280,  8360,  8440,  8520,
        8600,  8680,  8760,  8840,  8920,  9000,  9080,  9160,  9240,
        9320,  9400,  9480,  9560,  9640,  9720,  9800,  9880,  9960,
       10040, 10120, 10200, 10280, 10360, 10440, 10520, 10600, 10680,
       10760, 10840,

In [26]:
quot_10_19 = recuperer_donnees_gz('da6cd598-498b-4e39-96ea-fae89a4a8a46', 'QUOT_2010-2019', ';')
quot_10_19.tail()

,LAMBX,LAMBY,DATE,PRENEI,PRELIQ,T,FF,Q,DLI,SSI,...,RESR_NEIGE6,HTEURNEIGE,HTEURNEIGE6,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H
36125579,11960,17450,20191227,0.0,0.0,11.0,3.0,6.396,2714.8,503.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.328,0.0,9.6,15.0
36125580,11960,17450,20191228,0.0,0.0,9.7,2.3,4.937,2630.0,504.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.327,0.0,6.9,14.1
36125581,11960,17450,20191229,0.0,0.0,7.7,3.6,3.942,2371.0,841.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.326,0.0,5.8,12.9
36125582,11960,17450,20191230,0.0,0.0,9.6,4.5,3.653,2338.6,972.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.324,0.0,6.0,12.9
36125583,11960,17450,20191231,0.0,0.0,7.9,1.7,4.301,2418.4,849.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.322,0.0,4.4,12.8


In [27]:
quot_10_19["YEAR"] = quot_10_19["DATE"].astype(str).str[:4].astype(int)

In [ ]:
quot_10_19[quot_10_19["YEAR"]==2011]["LAMBX"].unique()

142

Il faudra repliquer la même chose sur toutes les bases depuis 1958, et fusionner toutes les bases en une seule complète de 1958 à 2025

## SWI Uniforme

In [ ]:
import os

# chemin vers le dossier 
folder_path = "C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform"

dfs = []

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        #print("Lecture :", file_path)
        df = pd.read_csv(file_path, sep=";")
        dfs.append(df)

swi_uniform_df = pd.concat(dfs, ignore_index=True)

print("Fusion terminée")
print(swi_uniform_df.shape)

#swi_uniform_df.to_csv("merged.csv", index=False)


Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.0-249.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.1000-1249.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.1250-1499.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.1500-1749.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.1750-1999.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.2000-2249.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform\swi.2250-2499.csv
Lecture : C:/Users/lisaw/Desktop/ENSAE/3A/Machine_le

In [16]:
swi_uniform_df.head()

,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS
0,2,641374,7106309,196001,"0,863"
1,2,641374,7106309,196002,"0,876"
2,2,641374,7106309,196003,"0,856"
3,2,641374,7106309,196004,"0,757"
4,2,641374,7106309,196005,"0,673"


In [19]:
swi_uniform_df[swi_uniform_df["LAMBX"]==11960]

,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS


## Données BRGM

In [6]:
# Base de données BRGM

def recuperer_donnees_7z(dataset_id, fichier_csv, sep):
    """
    Cette fonction télécharge les données d'un fichier CSV.GZ à partir d'une URL et les charge dans un DataFrame pandas.
    
    Paramètres:
    dataset_id (str): identifiant du fichier CSV.GZ.
    fichier_csv (str): Le nom du fichier CSV.GZ à enregistrer localement.
    sep ( str ) : le séparateur à utiliser
    
    Retourne:
    DataFrame: Un DataFrame pandas contenant les données du fichier CSV.
    """

    # URL de base pour accéder à l'API
    base_api = "https://www.data.gouv.fr/api/1/"

    # Chemin pour accéder aux enregistrements du dataset
    key_api = "datasets/r/"

    # Construction de l'URL complète
    url = f"{base_api}{key_api}{dataset_id}"


    response = requests.get(url)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        df = pd.read_csv(gz, sep = sep)
    
    return df

In [36]:
import os
import requests
import py7zr
import pandas as pd
from io import BytesIO
import tempfile

def load_7z_to_dataframe(url: str, csv_only=True):
    """
    Télécharge un fichier .7z depuis une URL, l'extrait,
    puis concatène les CSV trouvés en un DataFrame unique.
    
    Paramètres
    ----------
    url : str
        Lien de téléchargement du fichier .7z
    csv_only : bool
        Si True, charge uniquement les .csv
        
    Retour
    ------
    pd.DataFrame
    """
    
    # === 1) Télécharger le fichier ===
    print("Téléchargement du fichier .7z...")
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception("❌ Erreur de téléchargement :", response.status_code)
    
    # Stocker dans un buffer mémoire
    file_content = BytesIO(response.content)
    
    # === 2) Créer un dossier temporaire ===
    tmp_dir = tempfile.mkdtemp()
    
    # === 3) Extraire le fichier .7z ===
    print("Extraction...")
    with py7zr.SevenZipFile(file_content, mode='r') as z:
        z.extractall(path=tmp_dir)
    
    print(f"✅ Extraction dans : {tmp_dir}")

    # === 4) Charger tous les fichiers CSV en DataFrame ===
    dfs = []
    for root, _, files in os.walk(tmp_dir):
        for f in files:
            if not csv_only or f.endswith(".csv"):
                filepath = os.path.join(root, f)
                print("Lecture :", filepath)

                try:
                    df = pd.read_csv(filepath)
                    dfs.append(df)
                except Exception as e:
                    print(f"Erreur lecture {f} → ignoré : {e}")

    if not dfs:
        raise Exception("❌ Aucun CSV trouvé dans l’archive")

    # === 5) Fusionner en un DataFrame unique ===
    final_df = pd.concat(dfs, ignore_index=True)
    print("✅ Fusion terminée !")

    return final_df


In [37]:
url = "https://www.data.gouv.fr/api/1/datasets/r/c944be1e-06d6-46be-bf7d-9f9ad2b8ced9"

df = load_7z_to_dataframe(url)
print(df.head())


Téléchargement du fichier .7z...
Extraction...
✅ Extraction dans : C:\Users\lisaw\AppData\Local\Temp\tmp88dpvy17


Exception: ❌ Aucun CSV trouvé dans l’archive

In [7]:
data_BRGM = recuperer_donnees_gz('c944be1e-06d6-46be-bf7d-9f9ad2b8ced9', 'BRGM_argile', ';')
data_BRGM.tail()

BadGzipFile: Not a gzipped file (b'7z')